In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [2]:
df = pd.read_excel('Salad.xlsx')
df

,食材,营养素,颜色,韧性,口味,种类,分数
0,小番茄,纤维素,红,中,酸,蔬菜,10
1,生菜,纤维素,绿,低,NaN,蔬菜,10
2,卷心菜,纤维素,白,低,NaN,蔬菜,10
3,黄瓜,纤维素,绿,低,NaN,蔬菜,10
4,西蓝花,纤维素,绿,中,NaN,蔬菜,10
5,茭白,纤维素,白,中,NaN,蔬菜,10
6,竹笋,纤维素,白,中,NaN,蔬菜,10
7,芦笋,纤维素,绿,中,NaN,蔬菜,10
8,香菇,纤维素,褐,中,NaN,蔬菜,10
9,蟹味菇,纤维素,白,中,NaN,蔬菜,10


In [3]:
df0 = pd.read_excel('Salad.xlsx')
df = df0.copy()


def split_columns(value, value_range, suffix = ""):
    s = Series('', index = value_range)
    
    for x in value.split("，"):
        s.loc[suffix + x] = 1
    return s

def funcE(df, catolog):
    value_range = []
    suffix=catolog+'_'
    df.loc[pd.isnull(df[catolog]) == True,catolog]='空'
    
    for x in list(df[catolog].drop_duplicates()):
        value_range.extend(x.split('，'))
    value_range = Series(value_range).drop_duplicates().tolist()
    new_columns = [suffix+x for x in value_range]
    
    df[new_columns] = df[catolog].apply(lambda x:split_columns(x, new_columns, suffix))
    df = df.drop(catolog,axis=1)

    return df, value_range

In [4]:
def v_range(df):
    Cato=['种类','颜色','口味','韧性','营养素']
    Relationship = {}

    
    for x in Cato:
        value_range = []
        for y in df[x]:   
            
            if pd.notnull(y):
                value_range.extend(y.split('，'))
                
                AA=Series(value_range).drop_duplicates()
                
        Relationship[x]=list(AA)

    return Relationship

In [5]:
def update0(mt,cato_all,cato_consist):
    for x in cato_all:
        
        if mt[x].any() == 1:
            
            cato_consist.append(x)
                       
    a = Series(cato_consist).drop_duplicates()
    cato_consist = list(a)
    
    return cato_consist

In [6]:
def SeleF(df,input_mt):
    Cato=['种类','颜色','口味','韧性','营养素']
    Relationship=v_range(df0)
    cato_all = ['营养素_蛋白质','营养素_纤维素','营养素_脂肪','颜色_红','颜色_黄','颜色_绿']
    cato_consist = []
    SIGN1 = 0
    SIGN0 = 0
    result = []
    
    
    
    initiate(df,input_mt,cato_all,cato_consist)
    
    for x in Cato:
        for y in Relationship[x]:
            
            for a in input_mt:
                i_mt = df0[df0['食材'] == a]

                for b in i_mt.iloc[0]:

                    if b == y:   
                        SIGN0 = 2
            if SIGN0 == 2: 
                SIGN0 = 0
            else:
                mt = choose_one_max(df,y,x)

                result.append( list (mt['食材'])[0])

                df = change_data(df,mt)
                a = update0(mt,cato_all,cato_consist)

                if set(a) == set(cato_all):

                    SIGN1 = 1

                    break
        if SIGN1 == 1:
            break
    return result

In [7]:
def initiate(df,input_mt,cato_all,cato_consist):
    
    for x in input_mt:
        i_mt = df[df['食材'] == x]
        a = update0(i_mt,cato_all,cato_consist) 
        df = change_data(df,i_mt)


In [8]:
def choose_one_max(df,sub_cato,Cato):
    a = df[df[Cato + '_' + sub_cato]==1]

    return a[a['分数']==a['分数'].max()].sample(n=1)
    

In [9]:
def change_data(df,mt):
    cato_all = ['营养素_蛋白质','营养素_纤维素','营养素_脂肪','颜色_红','颜色_黄','颜色_绿']
    
    cato_consist = []
    mt_name = list(mt['食材'])[0]
    
    update1(mt,cato_all,cato_consist)
    cato_consist = update1(mt,cato_all,cato_consist)

    for x in cato_consist:
        if x != '营养素_蛋白质':
            df.loc[df[x] == 1,'分数'] = df.loc[df[x] == 1,'分数'] - 1
        else:
            df.loc[df[x] == 1,'分数'] = df.loc[df[x] == 1,'分数'] - 5
        
        
        break
    df = df[df['食材'] != mt_name]
        
    return df

In [10]:
def update1(mt,cato_all,cato_consist):
    for x in cato_all:
        
        if mt[x].any() == 1:
            
            cato_consist.append(x)
               
    b = Series(cato_consist).drop_duplicates()
    a=list(b)
    [a.remove(x) for x in ['食材','分数','口味_空','种类_空'] if x in a]
    cato_consist = a

    return cato_consist

In [11]:
for catlog in ['种类','颜色','口味','韧性','营养素']:    
    funcE(df, catlog)

In [12]:
input_mt = ['小番茄','西蓝花','甜玉米']
out_mt = SeleF(df,input_mt)
material_list = out_mt + input_mt
result_df = df.set_index('食材').reindex(material_list)

/home/exolution/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
def dict_to_string(d):
    string = ''
    for k, v in d.items():
        if v>0:
            string += k+'x'+str(v)+' '
    return string

In [14]:
def print_string(result_df, suffix, value_list):
    d = {}
    for value in value_list:
        d[value] = result_df[suffix+'_'+value].replace('',0).sum()
    print(suffix+':')
    print(dict_to_string(d))


print('预设食材:\n'+",".join(input_mt))
print('\n')

print('选取食材:\n'+",".join(out_mt))
print('\n')

print('最终食材组合:\n'+",".join(material_list))
print('\n')
    
param = v_range(df0)
param

for k,v in param.items():
    print_string(result_df, k, v)
    print('\n')


预设食材:
小番茄,西蓝花,甜玉米


选取食材:
橙肉,煎鸡胸(切丁),葡萄干,芝麻


最终食材组合:
橙肉,煎鸡胸(切丁),葡萄干,芝麻,小番茄,西蓝花,甜玉米


韧性:
中x6 高x1 


种类:
水果x1 蔬菜x2 粮食x1 辅料x2 肉类x1 


口味:
香x1 甜x3 咸x1 酸x1 


营养素:
蛋白质x1 纤维素x3 碳水化合物x2 脂肪x1 


颜色:
白x2 黄x2 褐x1 绿x1 红x1 


